In [31]:
# Imports we've seen before
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [32]:
# New imports
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [33]:
## VALIDATION SET APPROACH ##

# In this section we use the validation set approach to estimate the test error rates that arise
# from fitting various linear models on the Auto dataset

Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [34]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [35]:
X_valid = hp_mm.fit_transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
# validation MSE (estimate for test MSE)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [36]:
# We now estimate the validation error for higher-degree polynomial regressions
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred)**2)

In [37]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [38]:
# We now observe the influence the random split has on the estimated test MSE's:

Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

# Hence, our overall finding that a quadratic regression fit works best is upheld. However, we see
# that there is variation in the estimated MSE's due to the randomness of the train test split.

array([20.75540796, 16.94510676, 16.97437833])

In [39]:
## CROSS-VALIDATION ##

# Suppose we have a function A, and a function B, with some data D.
# If we want to compute B(A(D)), but A and B don't communicate with each other (i.e. the
# outut of A is not a valid input for B), then we need to use a *wrapper*.
# For this example, we use the sklearn_sm() wrapper which will allow us to use models from
# statsmodels and cross validation from sklearn.

hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

# The cv argument corresponds to the integer choice for K, which is the number of observations - hence
# corresponding to LOOCV.

24.23151351792924

In [40]:
# We use a for loop to test different degrees of polynomial fits:

cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i,d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

In [41]:
# Above, we introduced the outer() method of the np.power() function.
# outer() is applied to an operation that has two arguments, such as add() or power(),
# and it takes as input two arrays. It then returns a larger array where the given operation
# has been performed for each pair of elements in the two arrays.

A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In [42]:
# Cross validation with K=10:

cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0) # ensures the same splits for each degree
for i,d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

# We see that the computation time is (expectedly) shorter than for LOOCV.
# (However, in general, LOOCV should be faster for linear models if the equation shortcut is used)

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

In [43]:
# The cross-validate function is flexible, and can take various splitting mechanisms as an argument.
# For example, we can use the ShuffleSplit() function to easily implement the validation set approach.
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation);
results['test_score']

array([23.61661707])

In [44]:
# We estimate the variability in the test error:
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)
results['test_score'].mean(), results['test_score'].std()

# We note that this is not a valid estimate of the standard deviation of the calculated test scores,
# since the randomly shuffled training samples overlap, and hence have correlated results and hence
# correlated test scores.

(23.802232661034168, 1.421845094109185)

In [45]:
## THE BOOTSTRAP ##

# Using the portfolio dataset, we seek to estimate the parameter alpha we used in the example of 
# the bootstrap in Chapter 5.

Portfolio = load_data('Portfolio')

def alpha_func(D, idx):
    cov_ = np.cov(D[['X', 'Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] -  cov_[0,1]) / (cov_[0,0] + cov_[1,1] - 2*cov_[0,1]))

In [46]:
# We predict alpha using all 100 observations in the dataset
alpha_func(Portfolio, range(100))

0.57583207459283

In [47]:
# We construct a new bootstrap dataset by randomly selecting 100 observations from range(100)
# with replacement, and computing a new estimate for alpha
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619004

In [48]:
# A function that computes the standard error for any arbitrary function, func, where func takes
# a data frame and index as arguments and returns some value.

def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0,0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [49]:
# Use the function to evaluate our estimate for alpha using B=1000 bootstrap replications

alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277699

In [64]:
# Let's use bootstrapping to estimate the variability of the slope and intercept terms
# for a linear regression fit:

# Function for bootstrapping a regression model that uses a formula to define the corresponding
# regression
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [65]:
# For boot_SE, we expect only D and idx as inputs...
# To make boot_OLS usable in boot_SE, we can call partial() from functools to "freeze" the first
# two arguments of boot_OLS

hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [66]:
# We demonstrate hp_func's utility on the first ten bootstrap samples

rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])

KeyError: "None of [Index([333, 249, 200, 105, 120,  16,  29,   6,  68, 318,\n       ...\n       306, 206, 294, 205, 120,  34, 205, 384,  14, 223],\n      dtype='object', name='name', length=392)] are in the [index]"

In [74]:
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se

# We see the bootstrap estimates for the standard error of the intercept and horsepower coefficient:

intercept     0.731176
horsepower    0.006092
dtype: float64

In [70]:
# Let's compare these estimates to those from the standard formulas from section 3.1.2,
# which we can find using the ISLP.sm summarize() function:

hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

# We see that our estimates are pretty good! In fact, since bootstrapping does not rely on the assumption
# of linear data, its estimates are likely more accurate.

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [71]:
# Let's compare standard error estimates that result from applying a quadratic model
# through linear regression:

# Bootstrap estimates:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [72]:
# Standard errors computed using sm.OLS():

M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64